# API Testing

It is optional yet recommend to utilize a Python virtual environment:

1. Create virtual environment:<br/>`python3 -m venv .venv`
1. Activate virtual environment:
    * Windows: `.\.venv\Scripts\activate`
    * Linux or macOS: `source ./.venv/bin/activate`


In [ ]:
!pip install requests

In [ ]:
import requests
import json

## Test Params

In [ ]:
# Deployment Details
deploy_num = ""
deploy_region = ""

# Optionally hardcoded document IDs, otherwise first 2x docs dynamically used
doc1 = ""
doc2 = ""

## Define API

In [ ]:
# Define the API endpoint
api_service = f"https://esgdocanalysis-{deploy_num}.{deploy_region}.cloudapp.azure.com"

# Define API GET endpoints
api_get_docs = f"{api_service}/DocumentManager/GetAllDocuments"
api_get_gaps = f"{api_service}/ESRS/GetAllESRSGapAnalysisResults"
api_get_benchmarks = f"{api_service}/ESRS/GetAllESRSBenchmarkResults"

# Define API POST endpoints
api_post_docreg = f"{api_service}/DocumentManager/RegisterDocument"
api_post_askdoc = f"{api_service}/DocumentManager/AskAgainstDocument"
api_post_gap = f"{api_service}/ESRS/ESRSGapAnalyzerOnQueue"
api_post_benchmark = f"{api_service}/ESRS/ESRSDisclosureBenchmarkOnQueue"

## Reusable Functions

In [ ]:
# GET request
def get_request(url):
  print(f"GET Request: {url}")
  try:
    response = requests.get(url)
    # print("GET Response:", response.status_code, response.text)
    # return response
    return json.loads(response.text)
  except requests.exceptions.RequestException as e:
    print("GET ERROR:", e)
    return None

# POST request with JSON body
def post_request_with_json(url, json_data={}):
  print(f"POST Request: {url}")
  try:
    response = requests.post(url, json=json_data)
    # print("POST JSON Response:", response.status_code, response.text)
    # return response
    return json.loads(response.text)
  except requests.exceptions.RequestException as e:
    print("POST ERROR:", e)
    return None

# # POST request with a JSON body and file binary
# def post_request_with_file(url, json_data={}, file_path=""):
#     print(f"POST with File Request: {url}")
#     try:
#         files = {
#             'fileBinary': (file_path, open(file_path, 'rb'))
#         }
#         response = requests.post(url, data=json_data, files=files)
#         print("POST File Response:", response.status_code, response.text)
#         return response
#     except requests.exceptions.RequestException as e:
#         print("POST FILE ERROR:", e)
#         return None

## Test GET Requests

### GET All Registered Docs

In [ ]:
# Test GET All Registered Documents
all_docs = get_request(api_get_docs)
print(json.dumps(all_docs, indent=4))

### GET All Gap Analysis Reports

In [ ]:
# Test GET All Gap Analysis Reports
all_gaps = get_request(api_get_gaps)
print(json.dumps(all_gaps, indent=4))

### GET All Benchmark Reports

In [ ]:
# Test GET All Benchmark Reports
all_benchmarks = get_request(api_get_benchmarks)
print(json.dumps(all_benchmarks, indent=4))

## Test POST Requests

### POST Register Document

In [ ]:
# For now, register docs via frontend app to ensure complete data.

### Fallback for Automatic Docs

If docs not set, use first 2 items in array, which are 2 most recently registered documents

In [ ]:
# Warn if doc IDs are not populated
# Could be set dynamically from the GET API response
if (doc1 == "" and doc2 == ""):
  print("Document IDs are empty, attempting to populate...\n")


  # Check if all_docs array is populated with 2 items
  if len(all_docs) > 0 and len(all_docs) >= 2:
    # Get the first 2x document IDs
    doc1 = all_docs[0]["documentId"]
    doc2 = all_docs[1]["documentId"]
    print(f"Dynamically set IDs:\n* {doc1}\n* {doc2}")
  
  else:
    print("No documents found. Please register documents first.")
else:
  print(f"Document IDs are hardcoded:\n* {doc1}\n* {doc2}")

### POST Ask Document

In [ ]:
if doc1 != "":
  post_ask = post_request_with_json(
    api_post_askdoc,
    json_data = {
      "DocumentIds" : [doc1],
      "Question" : "What were the top sustainability efforts that could have the biggest impact?"
    }
  )
  print(json.dumps(post_ask, indent=4))

### POST Gap Analysis

In [ ]:
if doc1 != "":
  post_gap = post_request_with_json(
    api_post_gap,
    json_data=  {
      "disclosureName": "Transition plan for climate change mitigation",
      "disclosureNumber": "E1-1 16",
      "JobOwner" : "Chief Sustainability Officer",
      "disclosureRequirement": "Disclosure of transition plan for climate change mitigation",
      "disclosureRequirementDetail" : "The undertaking shall disclose its transition plan for climate change mitigation",
      "disclosureAnnex": "A transition plan relates to the undertaking's efforts in climate change mitigation. When disclosing its transition plan, the undertaking is expected to provide a high-level explanation of how it will adjust its strategy and business model to ensure compatibility with the transition to a sustainable economy and with the limiting of global warming to 1.5°C in line with the Paris Agreement (or an updated inter national agreement on climate change) and the objective of achieving climate neutrality by 2050 with no or limited overshoot as established in Regulation (EU) 2021/1119 (European Climate Law), and where applicable, how it will adjust its exposure to coal, and oil and gas-related activities. ",
      "documentId": doc1
    }
  )
  print(json.dumps(post_gap, indent=4))

### POST Benchmark Report

In [ ]:
if doc1 != "" and doc2 != "":
  post_benchmark = post_request_with_json(
    api_post_benchmark,
    json_data={
      "jobName": "Test Doc1 vs Doc2 E1-1 16a Benchmark",
      "documentIds": [doc1, doc2],
      "jobOwner": "Chief Sustainability Officer",
      "disclosureNumber": "E1-1 16a",
      "disclosureName": "Transition plan for climate change mitigation",
      "disclosureRequirement": "Disclosure of transition plan for climate change mitigation",
      "disclosureRequirementDetail" : "The undertaking shall disclose its transition plan for climate change mitigation",
      "disclosureAnnex" : "A transition plan relates to the undertaking's efforts in climate change mitigation. When disclosing its transition plan, the undertaking is expected to provide a high-level explanation of how it will adjust its strategy and business model to ensure compatibility with the transition to a sustainable economy and with the limiting of global warming to 1.5°C in line with the Paris Agreement (or an updated inter national agreement on climate change) and the objective of achieving climate neutrality by 2050 with no or limited overshoot as established in Regulation (EU) 2021/1119 (European Climate Law), and where applicable, how it will adjust its exposure to coal, and oil and gas-related activities."
    }
  )
  print(json.dumps(post_benchmark, indent=4))